In [1]:
#--- data ---
# package: readxl
data = data.frame(readxl::read_xlsx("Fig1c_data.xlsx"))
ref.date = as.Date("2020-01-01")
# 
# data = readxl::read_xlsx("Fig2c_data.xlsx")
data$x.lb <- as.numeric(as.Date(data$x.lb)-ref.date)
data$x.ub <- as.numeric(as.Date(data$x.ub)-ref.date)
data$y <- as.numeric(as.Date(data$y)-ref.date)

In [28]:
# data: (x.lb, x.ub): lower and upper bounds of infectors symtpom onset dates
# y: symptom onset dates of infectee

#--- functions ---

#--- CDF of serial interval ---
p.Z  = function(z, gpar, lnpar) {
  
  #--- infectiousness, gamma distribution ---
  # gpar[1:2]: hyper-parameters (gamma)
  # x        : infection time of infectee w.r.t onset time of infector
  f.Xc = function(x, gpar) { dgamma(x, gpar[1], gpar[2]) }
  
  #--- incubation, from Li et al NEJM 2020 ---
  # lnpar[1:2]: hyper-parameter (logNormal)
  # y         : length of incubation period of infectee
  f.Y  = function(y, lnpar) { dlnorm(y, lnpar[1], lnpar[2]) }
  
  #--- convolution between incubation and infectiousness profile ---
  # gpar[3]: shift c days before symptom onset of infector
  # z      : length of serial interval
  f.Z = function(z, gpar, lnpar) {
    integrate(
      f = function(x, z, gpar, lnpar) { f.Y(z+gpar[3]-x, lnpar)*f.Xc(x, gpar) },
      lower = -Inf, 
      upper = Inf,
      z     = z,
      gpar  = gpar,
      lnpar = lnpar
    )$value
  } 
  f.Z2 = Vectorize(f.Z, vectorize.args = "z")
  
  #--- p.Z ---
  integrate(
    f = function(x, gpar, lnpar) { f.Z2(x, gpar, lnpar) },
    lower = -Inf,
    upper = z,
    gpar  = gpar,
    lnpar = lnpar
  )$value
}
p.Z2 = Vectorize(p.Z, vectorize.args = c("z"))


#--- logLikelihood for the observed serial intervals ---
# x.lb: lower bound of infectors symtpom onset dates
# x.ub: upper bound of infectors symtpom onset dates
# y   : symptom onset dates of infectee
# 0.5 : continuity correction
lli.fx = function(gpar, x.lb, x.ub, y, lnpar) {
  lli = log(p.Z2(y-(x.lb-0.5), gpar, lnpar) - p.Z2(y-(x.ub+0.5), gpar, lnpar))
#   return(lli)
  return(-sum(lli[!is.infinite(lli)]))
}

#--- incubation period ---
# from Li et al NEJM 2020
# lognormal mean = 5.2; 95% CI = c(4.1, 7.0)
ln.par1 = 1.434065
ln.par2 = 0.6612

In [24]:
p.Z(-2, c(2.3, 0.5, 2.5), c(ln.par1, ln.par2))

[1] 2.547052e-07

In [25]:
lli.fx(c(2.3, 0.5, 2.5),data[, "x.lb"], data[, "x.ub"], data[, "y"], c(ln.par1, ln.par2))

[1] -2.4355012 -2.3392764 -2.4355012 -2.4370760 -2.4355012 -2.4355012
 [7] -2.4355012 -2.3147326 -4.1291343 -3.3513046 -4.3102931 -2.3147326
[13] -2.7030131 -2.3147326 -2.7030131 -2.7030131 -2.7030131 -2.7030131
[19] -4.1291343 -3.8189658 -2.7433719 -3.3513046 -3.2059896 -2.3392764
[25] -4.8110965 -1.6223642 -2.4355012 -1.2659214 -2.3392764 -2.3392764
[31] -2.1702788 -3.2059896 -3.1319866 -2.3147326 -4.3102931 -2.4370760
[37] -2.2922980 -2.4370760 -2.7383794 -2.7030131 -2.4355012 -2.7030131
[43] -4.1291343 -2.7030131 -2.7383794 -2.4355012 -2.5731009 -2.2922980
[49] -3.1319866 -2.4355012 -2.1348300 -2.5731009 -3.8189658       -Inf
[55] -2.7383794 -2.3147326 -2.3147326 -2.4370760 -2.4370760 -2.2922980
[61] -1.2166394 -1.2166394 -0.1383942 -3.2059896 -2.4370760 -0.9580505
[67] -2.7030131       -Inf -2.4370760 -2.3147326 -2.5731009 -0.6480940
[73] -2.4355012 -0.9580505 -2.2922980 -2.3147326 -3.3513046

In [31]:
#
#--- estimation ---
#

inf.fit = optim(
  c(2, 0.5, 3.7), lli.fx, 
  x.lb = data[, "x.lb"], 
  x.ub = data[, "x.ub"],
  y    = data[, "y"],
  lnpar = c(ln.par1, ln.par2)
)

# inf.fit$par would give the estimated parameters (1st and 2nd parameters) for the infectiousness,
# and start of infectiousness before symptom onset (3rd parameter)

In [32]:
#
#--- results from fit ---
#

(inf.par = inf.fit$par)
inf.par[3]                           # shift c
(inf.par[1]-1)/inf.par[2] - inf.par[3]     # mode
pgamma(inf.par[3], inf.par[1], inf.par[2]) # proportion of pre-symptomatic transmission

[1] 16.711114  1.444575 11.020943

[1] 11.02094

[1] -0.1450015

[1] 0.4544048